# HateSonar Analysis

This notebook ingests the preprocessed data from `../interim/metadata` downloaded by `download_datasets.ipynb` and quantifies the levels of hate speech or offensive language in each of the emails.

Finally, the analyses are merged and saved as a single csv file that is pushed to remote storage.

In [5]:
import pandas as pd
import os
import datetime
import re
from pathlib import Path
from dotenv import load_dotenv
from hatesonar import Sonar

load_dotenv("../../.env")

import sys

sys.path.append("../..")
from src import utils

In [6]:
BASE_PATH = os.getenv("LOCAL_DATA_PATH", "../../data/")

In [7]:
df = utils.load_dataset(f"{BASE_PATH}/interim/text/")

In [8]:
df.head()

,Message-ID,Date,Body
0,<23f4b2992626d689b84a704a575d974cc794709e.came...,"Fri, 31 Jul 2020 18:41:49 -0600","['On Fri, 2020-07-31 at 19:26 +0100, Richard W..."
1,<CAB_b4sBOn9Bisre7D3pUrDmH9+3unoP5VaeRGi031ks3...,"Sat, 01 Aug 2020 11:07:52 +0800",['Jerry James <loganjerry(a)gmail.com> =E4=BA=...
2,<CAJP_izdx=xTviDd4piWMLvxua7Ti8wD81kwqFEB7ucbG...,"Sat, 01 Aug 2020 03:25:48 -0400","['libcroco was retired on Rawhide, but the lib..."
3,<rg3f65$16fd$1@ciao.gmane.io>,"Sat, 01 Aug 2020 12:12:21 +0200","['Hi,\n\nseeing the amount of fallout from LTO..."
4,<rg3fi2$ipa$1@ciao.gmane.io>,"Sat, 01 Aug 2020 12:18:41 +0200",['Neal Gompa wrote:\n> I think it does have va...


In [9]:
sonar = Sonar()

2021-03-15 19:58:33,653 [2390] WARNING  py.warnings: /opt/app-root/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)

2021-03-15 19:58:33,662 [2390] WARNING  py.warnings: /opt/app-root/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)

2021-03-15 19:58:34,239 [2390] WARNING  py.warnings: /opt/app-root/lib/python3.6/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)



In [10]:
def speech(n):
    #sonar = Sonar()
    t = sonar.ping(text=n)
    top = t["top_class"]
    hate = t["classes"][0]["confidence"]
    off = t["classes"][1]["confidence"]
    neither = t["classes"][2]["confidence"]
    return [top, hate, off, neither]

def get_val(val):
    return val[loc]

In [11]:
df["sonar"] = df['Body'].apply(speech)
loc = 0
df['Top']  = df["sonar"].apply(get_val)
loc = 1 
df['Hate Speech']  = df["sonar"].apply(get_val)
loc = 2 
df['Offensive Language']  = df["sonar"].apply(get_val)
loc = 3 
df['Neither']  = df["sonar"].apply(get_val)

## Upload results to S3

In [12]:
new_files = (
    (df, f"{BASE_PATH}/processed/hatesonar.csv"),
)

In [13]:
Path(f"{BASE_PATH}/processed").mkdir(parents=True, exist_ok=True)

In [14]:
df.to_csv(new_files[0][1], header=False)

In [15]:
if os.getenv("RUN_IN_AUTOMATION"):
    utils.upload_files(
        (f, f"processed/{Path(f).stem}/hatesonar.csv") for _, f in new_files
    )